In [36]:
import tushare as ts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,Activation

Using TensorFlow backend.


# Part 1 -model

In [ ]:
df = pd.read_csv(r'./Data/TRAINSET_STOCK.csv')
df_add = pd.read_csv(r'./Data/20190506_STOCK.csv')
df_test = pd.read_csv(r'./Data/20190514_STOCK.csv')
df_test1 = pd.read_csv(r'./Data/20190521_STOCK.csv')
df = pd.concat([df,df_add,df_test,df_test1])
df['vol_change'] = df.groupby(["ts_code"])['vol'].pct_change()
df['low_change'] = df.groupby(["ts_code"])['low'].pct_change()
df['high_change'] = df.groupby(["ts_code"])['high'].pct_change()
df['vol_change'] = df['vol_change'].fillna(0)
df['low_change'] = df['vol_change'].fillna(0)
df['high_change'] = df['vol_change'].fillna(0)
df =  df[['ts_code','trade_date','name','pct_change','vol_change','high_change','low_change','y']]
df = df[['pct_change','vol_change','high_change','low_change','y']]
dataset = df.values
x = []
y = []
for i in range(len(dataset)-15):
    x.append(dataset[i:(i+10),:5].tolist())
    y.append(dataset[(i+10):(i+15),4,np.newaxis].tolist())
xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=0)
xtrain, xtest, ytrain, ytest = np.array(xtrain),np.array(xtest),np.array(ytrain),np.array(ytest)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape
ytrain = np.reshape(ytrain, (ytrain.shape[0], ytrain.shape[1]))
ytest= np.reshape(ytest, (ytest.shape[0], ytest.shape[1]))
regressor = Sequential()
regressor.add(LSTM(units = 60, return_sequences = True, input_shape = (xtrain.shape[1], 5)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60,return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 5))
regressor.add(Activation('sigmoid'))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(xtrain, ytrain, epochs = 100, batch_size = 50)
predicted_stock_price = regressor.predict(xtest)
a = []
for i in range(len(ytest)):
    fpr, tpr, thresholds = roc_curve(ytest[i], predicted_stock_price[i], pos_label=1)
    auc(fpr, tpr)
    a.append(auc(fpr, tpr))
a=pd.DataFrame(a)
a=a.dropna()
a.mean()

In [28]:
def pre(code):
    df1 = ts.get_hist_data(code) 
    df1=df1.head(n=11)
    df1=df1.sort_index(ascending=True)
    df1=df1[['p_change','volume','high','low']]
    df1['y']=[1,0,0,1,1,0,0,1,0,0,1]
    df1['vol_change'] = df1['volume'].pct_change()
    df1['low_change'] = df1['low'].pct_change()
    df1['high_change'] = df1['high'].pct_change()
    dataset=df1.values
    x=dataset.tolist()
    x=np.array(x)
    x=np.reshape(x, (1,x.shape[0],x.shape[1]))
    predicted = regressor.predict(x)
    print(predicted)


In [2]:
import tushare as ts
def getdata(code):
    df = ts.get_hist_data(code)
    df.to_csv('./预测.csv')

In [25]:
df=ts.get_hist_data('600809')
df

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20
date,,,,,,,,,,,,,
2019-05-24,55.49,57.07,56.11,54.80,55162.90,0.33,0.59,56.948,57.813,56.814,61124.07,72216.02,85300.42
2019-05-23,57.00,57.00,55.78,54.77,62237.24,-1.29,-2.26,57.666,57.792,56.919,63030.95,75479.08,86155.76
2019-05-22,58.00,58.10,57.07,56.20,55221.82,-0.91,-1.57,58.732,57.560,57.057,72944.35,79103.76,85976.28
2019-05-21,57.30,58.77,57.98,56.39,67572.08,0.18,0.31,59.172,57.476,57.184,85557.66,81307.91,84938.79
2019-05-20,59.50,59.50,57.80,56.11,65426.29,-1.90,-3.18,58.820,57.104,57.286,81656.05,83810.74,84414.41
2019-05-17,61.11,61.90,59.70,59.20,64697.32,-1.41,-2.31,58.678,56.556,57.358,83307.97,90329.47,83745.68
2019-05-16,58.94,61.86,61.11,58.00,111804.25,1.84,3.10,57.918,56.329,57.308,87927.22,94890.36,83732.27
2019-05-15,56.90,59.80,59.27,56.60,118288.37,3.05,5.42,56.388,55.778,57.108,85263.17,100277.02,81664.70
2019-05-14,56.00,57.48,56.22,55.82,48064.00,-0.87,-1.52,55.780,55.601,56.989,77058.16,95453.47,79691.31


In [26]:
for arr in df.columns:
    if df[arr].dtype=='float64':
        df[arr]=df[arr].astype('float32')
df.shape
    

(598, 13)

In [27]:
df['name']='山西汾酒'
# df['name']=df['name'].astype('category')

In [38]:
df['time']=df.index
df['time']=pd.to_datetime(df['time'],format='%Y-%m-%d')

In [40]:
df.time=df.time.dt.date


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 598 entries, 2019-05-24 to 2016-11-23
Data columns (total 15 columns):
open            598 non-null float32
high            598 non-null float32
close           598 non-null float32
low             598 non-null float32
volume          598 non-null float32
price_change    598 non-null float32
p_change        598 non-null float32
ma5             598 non-null float32
ma10            598 non-null float32
ma20            598 non-null float32
v_ma5           598 non-null float32
v_ma10          598 non-null float32
v_ma20          598 non-null float32
name            598 non-null object
time            598 non-null object
dtypes: float32(13), object(2)
memory usage: 44.4+ KB


In [45]:
df.describe(percentiles=[0.01, 0.99])

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20
count,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000
mean,45.055687,46.125954,45.216824,44.195004,77466.765625,0.060535,0.201120,45.104370,44.956226,44.673248,77496.859375,77539.531250,77524.648438
std,12.669917,12.971179,12.677661,12.363404,35652.707031,1.349566,2.804784,12.669785,12.664392,12.667871,24800.912109,20875.798828,17440.199219
min,22.389999,22.639999,22.549999,22.219999,17565.810547,-6.540000,-10.000000,22.639999,22.639999,22.639999,27939.619141,34952.351562,45001.160156
1%,22.697901,23.276701,22.749700,22.409700,22967.772793,-3.829600,-7.886000,22.869461,22.844939,22.844789,34435.543379,39151.050391,45550.320742
50%,46.699999,47.795000,46.965000,45.809999,70019.304688,-0.020000,-0.050000,46.481001,46.113998,45.692001,75610.722656,77747.910156,79319.398438
99%,65.052403,66.707497,65.284199,63.593900,187148.151406,3.515700,7.814200,65.153921,64.019103,63.602691,152463.169844,137956.057969,121225.475859
max,66.769997,68.000000,67.110001,65.779999,256190.312500,5.020000,10.010000,65.837997,65.414001,63.808998,176122.312500,149471.437500,128945.078125
